In [1]:
# import libraries
import pandas as pd
import scipy.stats as sts
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd
import osmnx as ox
import json
from datetime import datetime, timedelta, date
import os
import time
import pyarrow
import sys
import sidetable
import warnings
warnings.filterwarnings('ignore')

In [2]:
sys.tracebacklimit = 0 # turn off long tracebacks on error messages
#sys.tracebacklimit = None # use to turn tracebacks back on if needed

In [41]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [110]:
nh_all=pd.read_parquet('data/nh_sub_Jun27_21_Jul10_22_concat.parquet.gzip')

In [111]:
nh_all.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time
0,2021-06-27,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0,0,0,57,54,0,N,N,N,0,0,54,43,2,79.63,81.132,87,54,32,62.069,62.069
1,2021-06-27,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0,0,0,85,78,0,N,N,N,0,0,79,65,13,82.278,82.278,107,36,71,33.645,33.645
2,2021-06-27,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0,0,0,50,41,0,N,N,N,0,0,44,41,2,93.182,93.182,70,44,,62.857,62.857
3,2021-06-27,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0,0,0,90,65,0,N,N,N,0,0,68,64,,94.118,94.118,91,62,,68.132,68.132
4,2021-06-27,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0,1,0,103,76,0,N,N,N,0,0,79,67,,84.81,84.81,106,70,,66.038,66.038


In [112]:
nh_all.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time
836807,2022-07-10,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,Y,0,0,0,112,31,0,,,,0,0,31,30,1,96.774,96.774,83,79,4,95.181,95.181
836808,2022-07-10,745002,KENT COUNTY NURSING HOME,1443 NORTH MAIN,JAYTON,TX,79528,8062373036,Kent,N,,,,,,,,,,,,,,,,,,,,,,
836809,2022-07-10,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,N,0,0,0,12,1,0,,,,0,0,1,0,1,0,0,12,10,2,83.333,83.333
836810,2022-07-10,745005,CENTER AT ZARAGOZA,12660 PEBBLE HILLS BLVD.,EL PASO,TX,79938,9159901700,El Paso,N,,0,0,0,80,7,0,,,,0,,,,,,,,,,,
836811,2022-07-10,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,N,,,,,,,,,,,,,,,,,,,,,,


In [113]:
nh_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836812 entries, 0 to 836811
Data columns (total 32 columns):
 #   Column                                                                                                                                                          Non-Null Count   Dtype 
---  ------                                                                                                                                                          --------------   ----- 
 0   week_ending                                                                                                                                                     836812 non-null  object
 1   federal_provider_number                                                                                                                                         836812 non-null  object
 2   provider_name                                                                                                                   

In [114]:
#check for dup rows
nh_all.loc[nh_all.duplicated(),]

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time


In [115]:
# rename long cols
nh_all = nh_all.rename({'provider_zip_code':'provider_zip',
                                'passed_quality_assurance_check':'data_passed_qa',
                                'residents_weekly_confirmed_covid_19':'res_wk_confirmed_c19',
                                'number_of_all_beds':'num_all_beds',
                                'total_number_of_occupied_beds':'num_occ_beds',
                                'weekly_resident_confirmed_covid_19_cases_per_1_000_residents':'wk_res_confirmed_c19_cases_per_1000',
                                'number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result':'num_staff_new_pos_c19_test',
                                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week':'num_res_in_facility',
                                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time':'num_res_comp_c19_vax',
                                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time':'num_res_offered_declined_cvd_vax',
                                'percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time':'pct_res_comp_c19_vax',
                                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week':'num_hcp',
                                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time':'num_hcp_comp_c19_vax',
                                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time':'num_hcp_offered_declined_c19_vax',
                                'percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time': 'pct_hcp_comp_c19_vax'
                               }, axis=1)

In [116]:
nh_all.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,num_all_beds,num_occ_beds,staff_weekly_confirmed_covid_19,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,wk_res_confirmed_c19_cases_per_1000,num_staff_new_pos_c19_test,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time
0,2021-06-27,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0,0,0,57,54,0,N,N,N,0,0,54,43,2,79.63,81.132,87,54,32,62.069,62.069
1,2021-06-27,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0,0,0,85,78,0,N,N,N,0,0,79,65,13,82.278,82.278,107,36,71,33.645,33.645
2,2021-06-27,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0,0,0,50,41,0,N,N,N,0,0,44,41,2,93.182,93.182,70,44,,62.857,62.857
3,2021-06-27,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0,0,0,90,65,0,N,N,N,0,0,68,64,,94.118,94.118,91,62,,68.132,68.132
4,2021-06-27,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0,1,0,103,76,0,N,N,N,0,0,79,67,,84.81,84.81,106,70,,66.038,66.038


In [117]:
# subset cols
nh_sub = nh_all[['week_ending','federal_provider_number', 'provider_name', 'provider_address', 'provider_city', 'provider_state', 'provider_zip', 'provider_phone_number', 'county', 'submitted_data','data_passed_qa','num_all_beds','num_occ_beds','res_wk_confirmed_c19','wk_res_confirmed_c19_cases_per_1000','num_staff_new_pos_c19_test','num_res_in_facility','num_res_comp_c19_vax','num_res_offered_declined_cvd_vax','pct_res_comp_c19_vax','num_hcp','num_hcp_comp_c19_vax','num_hcp_offered_declined_c19_vax','pct_hcp_comp_c19_vax']]



In [118]:
nh_sub.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,num_all_beds,num_occ_beds,res_wk_confirmed_c19,wk_res_confirmed_c19_cases_per_1000,num_staff_new_pos_c19_test,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax
0,2021-06-27,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,57,54,0,0,0,54,43,2,79.63,87,54,32,62.069
1,2021-06-27,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,85,78,0,0,0,79,65,13,82.278,107,36,71,33.645
2,2021-06-27,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,50,41,0,0,0,44,41,2,93.182,70,44,,62.857
3,2021-06-27,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,90,65,0,0,0,68,64,,94.118,91,62,,68.132
4,2021-06-27,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,103,76,0,0,0,79,67,,84.81,106,70,,66.038


### revise data types

In [119]:
nh_sub=nh_sub.replace(r'^\s*$', np.nan, regex=True)
nh_sub.head(10)

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,num_all_beds,num_occ_beds,res_wk_confirmed_c19,wk_res_confirmed_c19_cases_per_1000,num_staff_new_pos_c19_test,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax
0,2021-06-27,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,57,54,0,0,0,54,43,2,79.63,87,54,32,62.069
1,2021-06-27,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,85,78,0,0,0,79,65,13,82.278,107,36,71,33.645
2,2021-06-27,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,50,41,0,0,0,44,41,2,93.182,70,44,NaN,62.857
3,2021-06-27,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,90,65,0,0,0,68,64,NaN,94.118,91,62,NaN,68.132
4,2021-06-27,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,103,76,0,0,0,79,67,NaN,84.81,106,70,NaN,66.038
5,2021-06-27,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,Limestone,Y,Y,149,122,0,0,0,122,85,21,69.672,141,77,58,54.61
6,2021-06-27,015019,MERRY WOOD LODGE,280 MT HEBRON ROAD,ELMORE,AL,36025,3345678484,Elmore,Y,Y,124,94,0,0,0,98,75,13,76.531,111,59,47,53.153
7,2021-06-27,015023,HATLEY HEALTH CARE INC,300 MEDICAL CENTER DRIVE,CLANTON,AL,35045,2057554960,Chilton,Y,Y,201,104,0,0,0,105,101,NaN,96.19,153,87,NaN,56.863
8,2021-06-27,015024,"LIMESTONE NURSING AND REHABILITATION CENTER, LLC",1600 WEST HOBBS STREET,ATHENS,AL,35611,2562323461,Limestone,Y,Y,170,122,0,0,0,124,103,NaN,83.065,152,65,NaN,42.763
9,2021-06-27,015027,"WETUMPKA HEALTH AND REHABILITATION, LLC",1825 HOLTVILLE ROAD,WETUMPKA,AL,36092,3345675131,Elmore,Y,Y,121,112,0,0,0,113,91,15,80.531,132,52,79,39.394


In [141]:
float_cols = ['num_all_beds','num_occ_beds','res_wk_confirmed_c19','wk_res_confirmed_c19_cases_per_1000','num_staff_new_pos_c19_test','num_res_in_facility','num_res_comp_c19_vax','num_res_offered_declined_cvd_vax','pct_res_comp_c19_vax','num_hcp','num_hcp_comp_c19_vax','num_hcp_offered_declined_c19_vax','pct_hcp_comp_c19_vax']

nh_sub[float_cols]=nh_sub[float_cols].astype('float')

nh_sub['week_ending'] = pd.to_datetime(nh_sub['week_ending'], format='%Y-%m-%d')

# subset to week ending greater than 7-11-21
nh_sub = nh_sub.loc[nh_sub['week_ending']>='2021-07-11']
nh_sub.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 806305 entries, 30507 to 836811
Data columns (total 26 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   week_ending                          806305 non-null  datetime64[ns]
 1   federal_provider_number              806305 non-null  object        
 2   provider_name                        806305 non-null  object        
 3   provider_address                     806305 non-null  object        
 4   provider_city                        806305 non-null  object        
 5   provider_state                       806305 non-null  object        
 6   provider_zip                         806305 non-null  object        
 7   provider_phone_number                806305 non-null  object        
 8   county                               806305 non-null  object        
 9   submitted_data                       806305 non-null  object      

In [162]:
# create week start date col
nh_sub['week_start'] = nh_sub['week_ending'] - timedelta(days=6)

In [163]:
nh_sub['month_year'] = pd.to_datetime(nh_sub['week_start']).dt.to_period('M')
nh_sub.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,num_all_beds,num_occ_beds,res_wk_confirmed_c19,wk_res_confirmed_c19_cases_per_1000,num_staff_new_pos_c19_test,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax,week_start,month_year
30507,2021-07-11,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,57.0,53.0,0.0,0.0,0.0,54.0,43.0,2.0,79.630,87.0,54.0,32.0,62.069,2021-07-05,2021-07
30508,2021-07-11,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,85.0,80.0,0.0,0.0,0.0,81.0,74.0,7.0,91.358,98.0,39.0,59.0,39.796,2021-07-05,2021-07
30509,2021-07-11,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,50.0,45.0,0.0,0.0,0.0,51.0,46.0,NaN,90.196,70.0,46.0,NaN,65.714,2021-07-05,2021-07
30510,2021-07-11,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,90.0,67.0,0.0,0.0,0.0,67.0,64.0,NaN,95.522,90.0,61.0,NaN,67.778,2021-07-05,2021-07
30511,2021-07-11,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,103.0,75.0,0.0,0.0,0.0,76.0,66.0,NaN,86.842,104.0,69.0,NaN,66.346,2021-07-05,2021-07


In [164]:
nh_sub.head(10)

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,num_all_beds,num_occ_beds,res_wk_confirmed_c19,wk_res_confirmed_c19_cases_per_1000,num_staff_new_pos_c19_test,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax,week_start,month_year
30507,2021-07-11,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,57.0,53.0,0.0,0.0,0.0,54.0,43.0,2.0,79.630,87.0,54.0,32.0,62.069,2021-07-05,2021-07
30508,2021-07-11,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,85.0,80.0,0.0,0.0,0.0,81.0,74.0,7.0,91.358,98.0,39.0,59.0,39.796,2021-07-05,2021-07
30509,2021-07-11,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,50.0,45.0,0.0,0.0,0.0,51.0,46.0,NaN,90.196,70.0,46.0,NaN,65.714,2021-07-05,2021-07
30510,2021-07-11,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,90.0,67.0,0.0,0.0,0.0,67.0,64.0,NaN,95.522,90.0,61.0,NaN,67.778,2021-07-05,2021-07
30511,2021-07-11,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,103.0,75.0,0.0,0.0,0.0,76.0,66.0,NaN,86.842,104.0,69.0,NaN,66.346,2021-07-05,2021-07
30512,2021-07-11,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,Limestone,Y,Y,149.0,121.0,0.0,0.0,0.0,123.0,90.0,31.0,73.171,141.0,78.0,57.0,55.319,2021-07-05,2021-07
30513,2021-07-11,015019,MERRY WOOD LODGE,280 MT HEBRON ROAD,ELMORE,AL,36025,3345678484,Elmore,Y,Y,124.0,90.0,0.0,0.0,0.0,96.0,75.0,14.0,78.125,96.0,54.0,35.0,56.250,2021-07-05,2021-07
30514,2021-07-11,015023,HATLEY HEALTH CARE INC,300 MEDICAL CENTER DRIVE,CLANTON,AL,35045,2057554960,Chilton,Y,Y,201.0,106.0,0.0,0.0,0.0,106.0,102.0,NaN,96.226,143.0,82.0,NaN,57.343,2021-07-05,2021-07
30515,2021-07-11,015024,"LIMESTONE NURSING AND REHABILITATION CENTER, LLC",1600 WEST HOBBS STREET,ATHENS,AL,35611,2562323461,Limestone,Y,Y,170.0,115.0,0.0,0.0,0.0,121.0,103.0,NaN,85.124,153.0,66.0,NaN,43.137,2021-07-05,2021-07
30516,2021-07-11,015027,"WETUMPKA HEALTH AND REHABILITATION, LLC",1825 HOLTVILLE ROAD,WETUMPKA,AL,36092,3345675131,Elmore,Y,Y,121.0,106.0,0.0,0.0,0.0,107.0,90.0,17.0,84.112,131.0,51.0,79.0,38.931,2021-07-05,2021-07


In [165]:
nh_sub.describe()

,num_all_beds,num_occ_beds,res_wk_confirmed_c19,wk_res_confirmed_c19_cases_per_1000,num_staff_new_pos_c19_test,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax
count,793626.000000,795001.000000,799170.000000,791806.000000,793455.000000,786738.000000,786738.000000,741889.000000,786738.000000,786758.000000,786296.000000,732184.000000,786758.000000
mean,106.263145,76.900593,0.570177,7.604106,0.820103,78.992759,68.228014,7.287551,86.487788,125.417404,100.256615,17.062326,77.677820
std,58.806072,48.351043,2.364547,32.866117,2.780067,48.472983,42.862642,8.515195,12.601002,82.828822,77.631436,21.374381,19.451309
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,65.000000,46.000000,0.000000,0.000000,0.000000,47.000000,40.000000,2.000000,81.579000,76.000000,53.000000,1.000000,65.546000
50%,99.000000,69.000000,0.000000,0.000000,0.000000,71.000000,60.000000,5.000000,89.286000,107.000000,82.000000,10.000000,81.944000
75%,127.000000,96.000000,0.000000,0.000000,1.000000,99.000000,85.000000,10.000000,94.891000,151.000000,124.000000,25.000000,94.118000
max,1119.000000,2689.000000,93.000000,5333.330000,388.000000,1545.000000,990.000000,305.000000,100.000000,7815.000000,1850.000000,437.000000,100.000000


In [166]:
nh_sub.data_passed_qa.value_counts()

Y    790349
N     10568
Name: data_passed_qa, dtype: int64

In [167]:
nh_dq_fail=nh_sub.loc[nh_sub['data_passed_qa']=='N']
nh_dq_fail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10568 entries, 30574 to 836809
Data columns (total 26 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   week_ending                          10568 non-null  datetime64[ns]
 1   federal_provider_number              10568 non-null  object        
 2   provider_name                        10568 non-null  object        
 3   provider_address                     10568 non-null  object        
 4   provider_city                        10568 non-null  object        
 5   provider_state                       10568 non-null  object        
 6   provider_zip                         10568 non-null  object        
 7   provider_phone_number                10568 non-null  object        
 8   county                               10568 non-null  object        
 9   submitted_data                       10568 non-null  object        
 10  data_

In [168]:
nh_dq_fail.provider_name.value_counts().sort_index()

ABBY GARDENS HEALTHCARE CENTER                         1
ABINGTON CREST HEALTHCAREANDREHABILITATION CENTER      1
ACCEL AT CRYSTAL PARK                                 53
ACCELA REHAB AND CARE CENTER AT SOMERTON               1
ACCORDIUS HEALTH AT BREVARD                            1
ACCORDIUS HEALTH AT CHARLOTTE                          2
ACCORDIUS HEALTH AT CONCORD                            1
ACCORDIUS HEALTH AT GASTONIA                           1
ACCORDIUS HEALTH AT HENDERSONVILLE LLC                 4
ACCORDIUS HEALTH AT ROSE MANOR LLC                     1
ACCORDIUS HEALTH AT WILMINGTON                         1
ACCURA HEALTHCARE OF SHENANDOAH                        1
ADVANCED CARE CENTER                                   2
ADVANCED HEALTH CARE OF AURORA                         1
ADVANCED HEALTH CARE OF COEUR D'ALENE LLC              1
ADVANCED HEALTH CARE OF SUMMERLIN                     53
ADVANCED REHABILITATION AND HEALTHCARE OF BOWIE        7
ADVENTHEALTH CARE CENTER APOPKA

In [169]:
nh_dq_pass=nh_sub.loc[nh_sub['data_passed_qa']=='Y']
nh_dq_pass.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,num_all_beds,num_occ_beds,res_wk_confirmed_c19,wk_res_confirmed_c19_cases_per_1000,num_staff_new_pos_c19_test,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax,week_start,month_year
30507,2021-07-11,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,57.0,53.0,0.0,0.0,0.0,54.0,43.0,2.0,79.630,87.0,54.0,32.0,62.069,2021-07-05,2021-07
30508,2021-07-11,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,85.0,80.0,0.0,0.0,0.0,81.0,74.0,7.0,91.358,98.0,39.0,59.0,39.796,2021-07-05,2021-07
30509,2021-07-11,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,50.0,45.0,0.0,0.0,0.0,51.0,46.0,NaN,90.196,70.0,46.0,NaN,65.714,2021-07-05,2021-07
30510,2021-07-11,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,90.0,67.0,0.0,0.0,0.0,67.0,64.0,NaN,95.522,90.0,61.0,NaN,67.778,2021-07-05,2021-07
30511,2021-07-11,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,103.0,75.0,0.0,0.0,0.0,76.0,66.0,NaN,86.842,104.0,69.0,NaN,66.346,2021-07-05,2021-07


In [170]:
nh_dq_pass.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,num_all_beds,num_occ_beds,res_wk_confirmed_c19,wk_res_confirmed_c19_cases_per_1000,num_staff_new_pos_c19_test,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax,week_start,month_year
836803,2022-07-10,676499,SPRINGTOWN PARK REHABILITATION AND CARE CENTER,201 WILLIAMS WARD RD.,SPRINGTOWN,TX,76082,5124382630,Parker,Y,Y,120.0,92.0,1.0,10.87,0.0,104.0,84.0,20.0,80.769,91.0,84.0,7.0,92.308,2022-07-04,2022-07
836804,2022-07-10,686123,KENDALL LAKES HEALTH AND REHABILITATION CENTER,5280 SW 157 AVENUE,MIAMI,FL,33185,7864337400,Miami-Dade,Y,Y,150.0,150.0,0.0,0.00,1.0,149.0,115.0,32.0,77.181,266.0,248.0,15.0,93.233,2022-07-04,2022-07
836805,2022-07-10,686124,HARMONY HEALTH CENTER,9820 N KENDALL DRIVE,MIAMI,FL,33176,3052716311,Miami-Dade,Y,Y,203.0,198.0,1.0,5.05,0.0,198.0,159.0,37.0,80.303,224.0,205.0,14.0,91.518,2022-07-04,2022-07
836806,2022-07-10,745000,LAS ALTURAS DE PENITAS,414 LIBERTY BLVD.,PENITAS,TX,78576,9569970800,Hidalgo,Y,Y,130.0,60.0,1.0,16.67,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-04,2022-07
836807,2022-07-10,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,Y,112.0,31.0,0.0,0.00,0.0,31.0,30.0,1.0,96.774,83.0,79.0,4.0,95.181,2022-07-04,2022-07


In [171]:
nh_dq_pass.describe()

,num_all_beds,num_occ_beds,res_wk_confirmed_c19,wk_res_confirmed_c19_cases_per_1000,num_staff_new_pos_c19_test,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax
count,787249.000000,788611.000000,790319.000000,788003.000000,788356.000000,781841.000000,781841.000000,737546.000000,781841.000000,781866.000000,781422.000000,727981.000000,781866.000000
mean,106.396006,77.148379,0.574717,7.620027,0.818676,79.265157,68.544209,7.311163,86.905509,125.837949,100.731127,17.116374,77.986513
std,58.755418,48.212216,2.370817,32.867288,2.730472,48.295606,42.680161,8.504349,11.151784,82.132606,77.460690,21.361671,18.855244
min,3.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,65.000000,46.000000,0.000000,0.000000,0.000000,47.000000,41.000000,2.000000,81.818000,76.000000,54.000000,1.000000,65.789000
50%,99.000000,69.000000,0.000000,0.000000,0.000000,71.000000,60.000000,5.000000,89.362000,107.000000,82.000000,10.000000,82.051000
75%,127.000000,96.000000,0.000000,0.000000,1.000000,99.000000,85.000000,10.000000,94.915000,151.000000,125.000000,25.000000,94.175000
max,1119.000000,2689.000000,93.000000,5333.330000,388.000000,1545.000000,990.000000,305.000000,100.000000,1851.000000,1850.000000,437.000000,100.000000


In [172]:
nh_dq_pass.num_hcp_comp_c19_vax.sum()/nh_dq_pass.num_hcp.sum()

0.8000283346370437

In [173]:
#subset to ky
nh_ky=nh_sub.loc[nh_sub['provider_state']=='KY']
nh_ky.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,num_all_beds,num_occ_beds,res_wk_confirmed_c19,wk_res_confirmed_c19_cases_per_1000,num_staff_new_pos_c19_test,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax,week_start,month_year
35436,2021-07-11,185003,LAUREL HEIGHTS HOME FOR THE ELDERLY,208 WEST TWELFTH STREET,LONDON,KY,40743,6068644155,Laurel,Y,Y,160.0,141.0,0.0,0.0,0.0,141.0,134.0,7.0,95.035,190.0,154.0,34.0,81.053,2021-07-05,2021-07
35437,2021-07-11,185005,SPRING CREEK POST-ACUTE AND REHABILITATION CENTER,1401 SOUTH 16TH STREET,MURRAY,KY,42071,2707522900,Calloway,Y,Y,226.0,102.0,0.0,0.0,0.0,102.0,93.0,6.0,91.176,150.0,53.0,0.0,35.333,2021-07-05,2021-07
35438,2021-07-11,185006,MORGANTOWN CARE & REHABILITATION CENTER,201 SOUTH WARREN STREET,MORGANTOWN,KY,42261,2705263368,Butler,Y,Y,122.0,103.0,0.0,0.0,0.0,103.0,91.0,12.0,88.350,160.0,66.0,0.0,41.250,2021-07-05,2021-07
35439,2021-07-11,185008,OWENSBORO HEALTH MUHLENBERG COMMUNITY HOSPITAL...,440 HOPKINSVILLE STREET,GREENVILLE,KY,42345,2703388433,Muhlenberg,Y,Y,45.0,39.0,0.0,0.0,2.0,41.0,39.0,NaN,95.122,114.0,80.0,NaN,70.175,2021-07-05,2021-07
35440,2021-07-11,185012,HILLSIDE CENTER,1500 PRIDE AVENUE,MADISONVILLE,KY,42431,2708211813,Hopkins,Y,Y,71.0,53.0,0.0,0.0,0.0,52.0,48.0,4.0,92.308,80.0,42.0,34.0,52.500,2021-07-05,2021-07


In [174]:
# who failed dq check and how many for how many occurances (weeks)?
nh_ky.loc[nh_ky['data_passed_qa']=='N'].provider_name.value_counts()

BRECKINRIDGE MEMORIAL NURSING FACILITY                53
FLAGET MEMORIAL HOSPITAL NF                           53
KINGS DAUGHTERS MEDICAL CENTER                        53
ST CLAIRE MEDICAL CENTER                              42
NIM HENSON GERIATRIC CENTER                           17
RIVER HAVEN NURSING AND REHABILITATION CENTER         15
BAPTIST HEALTH HARDIN                                  8
FRANKLIN-SIMPSON NURSING AND REHABILITATION CENTER     7
SALYERSVILLE NURSING AND REHABILITATION CENTER         4
SIGNATURE HEALTHCARE OF HART COUNTY REHAB & WELLNE     4
FOUNTAIN CIRCLE CARE & REHABILITATION CENTER           3
WEST LIBERTY NURSING AND REHABILITATION                3
PARKVIEW POST-ACUTE AND REHABILITATION CENTER          3
SIGNATURE HEALTHCARE OF GLASGOW REHAB & WELLNESS C     2
LYNDON WOODS CARE & REHAB, LLC                         2
LANDMARK OF DANVILLE REHABILITATION AND NURSING CE     1
SANSBURY CARE CENTER                                   1
CLIFTON OAKS CARE AND REHAB CEN

In [175]:
nh_ky_pass=nh_ky.loc[nh_ky['data_passed_qa']=='Y']
nh_ky_pass.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14529 entries, 35436 to 826826
Data columns (total 26 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   week_ending                          14529 non-null  datetime64[ns]
 1   federal_provider_number              14529 non-null  object        
 2   provider_name                        14529 non-null  object        
 3   provider_address                     14529 non-null  object        
 4   provider_city                        14529 non-null  object        
 5   provider_state                       14529 non-null  object        
 6   provider_zip                         14529 non-null  object        
 7   provider_phone_number                14529 non-null  object        
 8   county                               14529 non-null  object        
 9   submitted_data                       14529 non-null  object        
 10  data_

In [176]:
nh_ky_pass.describe()

,num_all_beds,num_occ_beds,res_wk_confirmed_c19,wk_res_confirmed_c19_cases_per_1000,num_staff_new_pos_c19_test,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax
count,14512.000000,14512.000000,14529.000000,14508.000000,14497.000000,14422.000000,14422.000000,13735.000000,14422.000000,14419.000000,14414.000000,13539.000000,14419.000000
mean,95.513713,74.891262,0.691651,9.636018,0.899221,76.222091,66.070587,7.246305,86.479955,114.697344,84.011308,22.150011,73.270923
std,40.153758,33.947990,2.544275,36.099500,2.532593,31.593030,28.668554,6.257366,9.798259,58.090314,50.800421,23.524109,19.644456
min,10.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,64.000000,52.000000,0.000000,0.000000,0.000000,53.000000,45.000000,3.000000,82.075000,80.000000,54.000000,2.000000,58.824000
50%,95.000000,73.000000,0.000000,0.000000,0.000000,74.000000,63.000000,6.000000,88.571000,104.000000,74.000000,15.000000,76.224000
75%,120.000000,93.000000,0.000000,0.000000,1.000000,95.000000,83.000000,10.000000,93.056000,133.000000,101.000000,35.000000,90.000000
max,285.000000,1001.000000,43.000000,666.670000,71.000000,205.000000,198.000000,60.000000,100.000000,660.000000,593.000000,217.000000,100.000000


In [177]:
# subset to nelson county, ky
nh_ky_cty_nelson = nh_ky.loc[nh_ky.county=='Nelson'].reset_index(drop=True)
nh_ky_cty_nelson.sort_values(by=['provider_name','week_ending'])

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,num_all_beds,num_occ_beds,res_wk_confirmed_c19,wk_res_confirmed_c19_cases_per_1000,num_staff_new_pos_c19_test,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax,week_start,month_year
2,2021-07-11,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,12.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.000,0.0,0.0,NaN,0.000,2021-07-05,2021-07
5,2021-07-18,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,12.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.000,0.0,0.0,NaN,0.000,2021-07-12,2021-07
8,2021-07-25,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,12.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.000,0.0,0.0,NaN,0.000,2021-07-19,2021-07
11,2021-08-01,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,12.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,2021-07-26,2021-07
14,2021-08-08,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,12.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.000,0.0,0.0,NaN,0.000,2021-08-02,2021-08
17,2021-08-15,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,12.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.000,0.0,0.0,NaN,0.000,2021-08-09,2021-08
20,2021-08-22,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,12.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.000,0.0,0.0,NaN,0.000,2021-08-16,2021-08
23,2021-08-29,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,12.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.000,0.0,0.0,NaN,0.000,2021-08-23,2021-08
26,2021-09-05,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,12.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,2021-08-30,2021-08
29,2021-09-12,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,12.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.000,0.0,0.0,NaN,0.000,2021-09-06,2021-09


In [178]:
nh_ky_cty_nelson.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 26 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   week_ending                          159 non-null    datetime64[ns]
 1   federal_provider_number              159 non-null    object        
 2   provider_name                        159 non-null    object        
 3   provider_address                     159 non-null    object        
 4   provider_city                        159 non-null    object        
 5   provider_state                       159 non-null    object        
 6   provider_zip                         159 non-null    object        
 7   provider_phone_number                159 non-null    object        
 8   county                               159 non-null    object        
 9   submitted_data                       159 non-null    object        
 10  data_passed_qa

In [179]:
nh_ky_cty_nelson.provider_name.value_counts()

LANDMARK OF BARDSTOWN REHABILITATION AND NURSING     53
SIGNATURE HEALTHCARE AT COLONIAL REHAB & WELLNESS    53
FLAGET MEMORIAL HOSPITAL NF                          53
Name: provider_name, dtype: int64

In [185]:
nh_ky_cty_nelson.loc[(nh_ky_cty_nelson['month_year']>='2022-04') & (nh_ky_cty_nelson['data_passed_qa']=='Y')].sort_values(by=['provider_name'])

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,num_all_beds,num_occ_beds,res_wk_confirmed_c19,wk_res_confirmed_c19_cases_per_1000,num_staff_new_pos_c19_test,num_res_in_facility,num_res_comp_c19_vax,num_res_offered_declined_cvd_vax,pct_res_comp_c19_vax,num_hcp,num_hcp_comp_c19_vax,num_hcp_offered_declined_c19_vax,pct_hcp_comp_c19_vax,week_start,month_year
117,2022-04-10,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,100.0,79.0,0.0,0.00,0.0,79.0,66.0,10.0,83.544,63.0,50.0,10.0,79.365,2022-04-04,2022-04
138,2022-05-29,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,100.0,83.0,0.0,0.00,0.0,83.0,71.0,10.0,85.542,65.0,51.0,13.0,78.462,2022-05-23,2022-05
156,2022-07-10,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,100.0,81.0,0.0,0.00,4.0,81.0,68.0,11.0,83.951,65.0,51.0,14.0,78.462,2022-07-04,2022-07
135,2022-05-22,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,100.0,88.0,0.0,0.00,0.0,88.0,74.0,10.0,84.091,62.0,47.0,13.0,75.806,2022-05-16,2022-05
147,2022-06-19,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,100.0,83.0,0.0,0.00,0.0,83.0,71.0,10.0,85.542,67.0,53.0,13.0,79.104,2022-06-13,2022-06
132,2022-05-15,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,100.0,72.0,0.0,0.00,0.0,83.0,72.0,8.0,86.747,63.0,49.0,13.0,77.778,2022-05-09,2022-05
141,2022-06-05,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,100.0,80.0,0.0,0.00,0.0,80.0,69.0,9.0,86.250,65.0,51.0,13.0,78.462,2022-05-30,2022-05
129,2022-05-08,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,100.0,81.0,0.0,0.00,0.0,81.0,67.0,11.0,82.716,62.0,48.0,12.0,77.419,2022-05-02,2022-05
126,2022-05-01,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,100.0,80.0,0.0,0.00,0.0,80.0,67.0,10.0,83.750,61.0,47.0,12.0,77.049,2022-04-25,2022-04
123,2022-04-24,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,100.0,83.0,0.0,0.00,0.0,83.0,68.0,10.0,81.928,61.0,47.0,12.0,77.049,2022-04-18,2022-04


## Shapefile data

In [3]:
# Read in shp file as dataframe
# Nursing home shapefile data source: HIFLD data retrieved from: 
# https://hifld-geoplatform.opendata.arcgis.com/datasets/geoplatform::nursing-homes/about
# Latest update for files: Jun 29, 22

nh_shp_df = gpd.read_file('data/Nursing_Homes_shape/NursingHomes.shp')
nh_shp_df.head()

,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,...,TOT_RES,TOT_STAFF,BEDS,EXCESS_BED,OWNERSHIP,MEDICAIDID,MEDICAREID,STATE_LIC,SOURCETYPE,geometry
0,1,0235,HAINES ASSISTED LIVING,219 UNION STREET,HAINES,AK,99827,NOT AVAILABLE,(907) 766-3616,ASSISTED LIVING,...,10,-999,10,-999,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,100793,ASSISTED LIVING,POINT (-135.44587 59.23694)
1,2,0241,LAKEVIEW HOME,2675 WESLEYAN DRIVE,ANCHORAGE,AK,99508,NOT AVAILABLE,(907) 333-8921,ASSISTED LIVING,...,11,-999,11,-999,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,235,ASSISTED LIVING,POINT (-149.78833 61.19643)
2,3,0257,ALASKA VETERANS' AND PIONEERS' HOME,250 E. FIREWEED AVENUE,PALMER,AK,99645,NOT AVAILABLE,(907) 745-4241,ASSISTED LIVING,...,79,-999,79,-999,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,77,ASSISTED LIVING,POINT (-149.11363 61.59638)
3,4,0268,TRANQUILITY MANOR ESTATES,1950 HEMMER ROAD,PALMER,AK,99645,NOT AVAILABLE,(907) 746-4220,ASSISTED LIVING,...,17,-999,17,-999,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,100954,ASSISTED LIVING,POINT (-149.14849 61.60133)
4,5,0249,NORTHERN COMFORT,2800 N. LAGOON DRIVE,WASILLA,AK,99654,NOT AVAILABLE,(907) 746-6493,ASSISTED LIVING,...,16,-999,16,-999,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,241,ASSISTED LIVING,POINT (-149.28398 61.61176)
